In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime, time, random
import json, pickle
import requests

In [71]:
df = pd.read_excel("end_table-21_01_19.xls")

In [74]:
rates = pd.read_excel("rates.xlsx")

In [78]:
rates['Distance slab'] = np.arange(0, 470, 10)

In [11]:
rates['distance_lower_bound'] = rates['Distance slab'].apply(lambda x : x.split('-')[0])
rates['distance_upper_bound'] = rates['Distance slab'].apply(lambda x : x.split('-')[1])

In [80]:
rates_dict = {}

In [81]:
for row in rates.iterrows():
    rates_dict[float(row[1]['Distance slab'])] = row[1]
#     break
#     print(row['distance_lower_bound'])

In [86]:
rates_dict[0].keys()

Index(['Rate/Ton', 'Distance slab', 'Diesel price', 'Diesel/ton ',
       'Driver Expense'],
      dtype='object')

In [89]:
df['adjusted_distance_lower_bound'] = (df['adjusted_distance']//10)*10
df.head()

,From,Godown location,Consignee Location,distance,time_text,time_value,adjusted_distance,journey_distance,adjusted_distance_lower_bound
0,Dhanbad,Bhutgaria,Dhanbad(JH),11.027,28 mins,1662.0,16.5405,22.054,10.0
1,Dhanbad,Bhutgaria,Hazaribagh,131.985,2 hours 55 mins,10505.0,158.3820,263.970,150.0
2,Dhanbad,Bhutgaria,BHUDA,12.582,34 mins,2031.0,18.8730,25.164,10.0
3,Dhanbad,Bhutgaria,JC MALLICK ROAD,13.087,34 mins,2055.0,19.6305,26.174,10.0
4,Dhanbad,Bhutgaria,BISHNUPUR,263.360,6 hours 50 mins,24594.0,316.0320,526.720,310.0


In [42]:
rates_dict.keys()

dict_keys(['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '200', '210', '220', '230', '240'])

In [83]:
df.dropna(inplace = True)

In [92]:
# Rate/Ton          200.0
#  Distance slab       0.0
#  Diesel price      510.0
#  Diesel/ton         20.4
#  Driver Expense     32.0
rate_per_ton = []
diesel_price = []
diesel_per_ton = []
driver_expense = []

for i in df['adjusted_distance_lower_bound']:
#     print(rates_dict[i]['Rate/Ton'], rates_dict[i]['Diesel price'], rates_dict[i]['Diesel/ton '], rates_dict[i]['Driver Expense'])
    rate_per_ton.append(rates_dict[i]['Rate/Ton'])
    diesel_price.append(rates_dict[i]['Diesel price'])
    diesel_per_ton.append(rates_dict[i]['Diesel/ton '])
    driver_expense.append(rates_dict[i]['Driver Expense'])

In [95]:
df['Rate Per Ton'] = rate_per_ton
df['Diesel Price'] = diesel_price
df['Diesel Per Ton'] = diesel_per_ton
df['Driver Expense'] = driver_expense
df['Handling Charges'] = 0.2 * df['Rate Per Ton']

In [97]:
df.drop(columns = ['adjusted_distance_lower_bound']).to_excel("almost_done.xls")

In [64]:
df['adjusted_distance_lower_bound'].apply(lambda x : rates_dict[x] if not np.isnan(x) else {'hello':'world'})

IndexError: list assignment index out of range

In [ ]:
df['']

In [45]:
rates_dict.keys()

dict_keys(['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '200', '210', '220', '230', '240', 0.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 110.0, 120.0, 130.0, 140.0, 150.0, 160.0, 170.0, 180.0, 190.0, 200.0, 210.0, 220.0, 230.0, 240.0])

In [101]:
final_df = df.drop('adjusted_distance_lower_bound', axis = 1)

In [104]:
final_df['Cost w/o handling charges'] = final_df['Rate Per Ton'] + final_df['Diesel Per Ton'] + final_df['Driver Expense']

In [105]:
final_df['Cost inc handling Charges'] = final_df['Cost w/o handling charges'] + final_df['Handling Charges']

In [110]:
final_df.columns

Index(['From ', 'Godown location', 'Consignee Location', 'distance',
       'time_text', 'time_value', 'adjusted_distance', 'journey_distance',
       'Rate Per Ton', 'Diesel Price', 'Diesel Per Ton', 'Driver Expense',
       'Handling Charges', 'Cost w/o handling charges',
       'Cost inc handling Charges'],
      dtype='object')

In [111]:
polished_df = final_df[['From ', 'Godown location', 'Consignee Location', 'adjusted_distance', 'Cost w/o handling charges', 'Cost inc handling Charges']]

In [115]:
polished_df.sort_values(by = ['Godown location', 'adjusted_distance']).to_excel("cleaned_sorted.xls")